<a href="https://colab.research.google.com/github/shaoyinguo-portfolio/CorpGenie-exp/blob/main/Meeting2TechDoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
from time import time
from PIL import Image
import base64
import io


try:
    import gdown
except:
    !pip install gdown
    import gdown

In [2]:
drive.mount('/content/drive')
data_path = Path('/content/drive/MyDrive/Colab Notebooks/data')

TRANSCRIPT_PATH = f'{data_path}/transcripts.txt'
KEYFRAME_PATH = f'{data_path}/key_frames'

Mounted at /content/drive


In [3]:
all_lines = []

with open(TRANSCRIPT_PATH, 'r') as f:
    for line in f.readlines():
        # print(line)
        splits = line.split(']')
        if len(splits) < 2:
            continue
        ts = float(splits[0].strip().replace('[', ''))
        text = splits[1].strip()
        all_lines.append((ts, text))

In [9]:
import glob

all_images = []
for p in glob.glob('/content/drive/MyDrive/Colab Notebooks/data/key_frames/*.jpg'):
    all_images.append((float(Path(p).name.replace('.jpg', '')), p))

# all_images.sort(key=lambda x: x[0])

In [12]:
all_events = []
for ts, img in all_images:
    # Convert numpy array to PIL Image as required by the processor
    all_events.append({'timestamp': ts, 'type': 'image', 'data': img})

for ts, text in all_lines:
    all_events.append({'timestamp': ts, 'type': 'transcript', 'data': text})

# Sort events chronologically
all_events.sort(key=lambda x: x['timestamp'])

In [8]:
def encode_image_to_base64(image_path: str, format: str = 'PNG') -> str:
    """
    Reads an image file using PIL, saves it into an in-memory buffer,
    and encodes the buffer contents into a Base64 string.

    Args:
        image_path: The file path to the image.
        format: The format to use for the buffer (PNG is recommended for slides).
                Must be a format PIL supports.

    Returns:
        A Base64 encoded string of the image data.
    """
    try:
        # 1. Read the image file using PIL
        with Image.open(image_path) as img:
            # Ensure image is in RGB format if it's grayscale or otherwise different
            if img.mode != 'RGB':
                img = img.convert('RGB')

            # 2. Save the image data to an in-memory buffer (BytesIO)
            # This avoids writing a temporary file
            buffered = io.BytesIO()
            img.save(buffered, format=format)

            # 3. Encode the bytes from the buffer to Base64
            img_bytes = buffered.getvalue()
            img_base64 = base64.b64encode(img_bytes)

            # 4. Convert bytes to a string for the API payload
            return img_base64.decode('utf-8')

    except FileNotFoundError:
        print(f"Error: The file was not found at {image_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [15]:
# Build the API-specific 'messages' list
content_blocks = []
for item in all_events:
    hour = int(item['timestamp'] // 3600)
    minute = int((item['timestamp'] % 3600) // 60)
    second = int(item['timestamp'] % 60)
    time_str = f"{hour:02d}:{minute:02d}:{second:02d}"

    if item['type'] == 'image':
        base64_image = encode_image_to_base64(item['data'])
        content_blocks.append({
            "type": "image_url",
            "image_url": {
                # Send as base64 data URL
                "url": f"data:image/png;base64,{base64_image}"
            }
        })
        content_blocks.append({
            "type": "text",
            "text": f"[Timestamp {time_str}]: Visual slide content related to the ongoing discussion."
        })
    elif item['type'] == 'transcript':
        content_blocks.append({
            "type": "text",
            "text": f"[Timestamp {time_str}]: Transcript excerpt: '{item['data']}'"
        })


In [24]:
img_count = 0
for i, item in enumerate(content_blocks):
    if item['type'] == 'image_url':
        img_count += 1
    if img_count > 3:
        content_blocks_to_prompt = content_blocks[:i]
        break

In [26]:
messages_payload = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Using the provided meeting key frames and transcript, correct any potential mistakes in the transcript by strictly referring to the slides. Please write a final, accurate, detailed, and professional technical document. Focus on accurate metrics and key action items. Keep corresponding image tags using their timestamps in appropriate locations for resourcing, such as etc."
    },
    {
        "role": "user",
        "content": content_blocks_to_prompt
    }
]

In [27]:
import json
import sys

# Assume messages_payload is your final list of dictionaries ready to be sent
json_payload_str = json.dumps(messages_payload)
byte_size = len(json_payload_str.encode('utf-8'))

print(f"Estimated payload size: {byte_size / 1024 / 1024:.2f} MB")

# Check against a hypothetical limit (e.g., 50 MB)
if byte_size > 50 * 1024 * 1024:
    print("Warning: Payload might be too large for real-time API limits. Consider reducing the input.")


Estimated payload size: 3.45 MB


In [18]:
from google.colab import userdata
import os

# Retrieve the key and set it as an environment variable
# This is ideal if the library you use expects an environment variable
os.environ["OPENAI_API_KEY"] = userdata.get('CorpGenie')

# You can now use the environment variable safely
# For example, when initializing an OpenAI client
# client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [28]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.environ["OPENAI_API_KEY"],
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="google/gemini-2.0-flash-exp:free", #"qwen/qwen2.5-vl-32b-instruct:free",
  messages=messages_payload
)
print(completion.choices[0].message.content)

Okay, here is the corrected and detailed technical document, based on the provided meeting key frames and transcript:

**Technical Document: TSMC and Intel Packaging Technologies: CoWoS, EMIB, Foveros and Chiplets**

**Introduction**
This document provides an overview of advanced packaging technologies from TSMC and Intel, focusing on CoWoS, EMIB, Foveros, and chiplets. It also discusses the Key Performance Indicators (KPIs) driving developments in packaging technology.

[Timestamp 00:00:01]
**TSMC and Intel: Packaging Technology Overview**
TSMC is currently the leader in semiconductor manufacturing, not solely due to high yields at 3nm and 2nm process nodes, but also due to its advanced packaging capabilities, including its 3D fabrics. While TSMC's process encompasses more than CoWoS, this presentation initially focuses specifically on CoWoS. Intel offers EMIB and Foveros as alternative technologies. The document will address chiplets, outlining the advantages and disadvantages associ

Okay, here is the corrected and detailed technical document, based on the provided meeting key frames and transcript:

**Technical Document: TSMC and Intel Packaging Technologies: CoWoS, EMIB, Foveros and Chiplets**

**Introduction**
This document provides an overview of advanced packaging technologies from TSMC and Intel, focusing on CoWoS, EMIB, Foveros, and chiplets. It also discusses the Key Performance Indicators (KPIs) driving developments in packaging technology.

[Timestamp 00:00:01]
**TSMC and Intel: Packaging Technology Overview**
TSMC is currently the leader in semiconductor manufacturing, not solely due to high yields at 3nm and 2nm process nodes, but also due to its advanced packaging capabilities, including its 3D fabrics. While TSMC's process encompasses more than CoWoS, this presentation initially focuses specifically on CoWoS. Intel offers EMIB and Foveros as alternative technologies. The document will address chiplets, outlining the advantages and disadvantages associated with their implementation.

**KPIs for Packaging Technology (P3C2)**
[Timestamp 00:01:30]
The core KPIs for packaging technology are often referred to as P3C2, a concept highlighted by Douglas of TSMC. P3C2 encompasses:

*   **Performance:** Bandwidth (BW), Fmax, Functionality
*   **Power:** Efficiency, Thermal management (Tj)
*   **Profile:** Footprint, Thickness
*   **Cycle time:** Structure, Process Flow
*   **Cost:** Material, Yield
    These factors collectively influence heterogeneous system integration and drive the industry towards continued pitch scaling. It's important to note that while customers prioritize the aforementioned factors, process technology development heavily relies on yield and reliability as key decisive factors.

Action Items:

*   Further explore TSMC's 3D fabrics in future analysis.
*   Elaborate on the specific applications and advantages of EMIB and Foveros versus CoWoS.
*   Provide a detailed breakdown of cost analysis related to different packaging technologies.
*   Investigate and quantify the impact of yield and reliability on the overall P3C2 framework.